In [ ]:
#try: # Install microgrids package in JupyterLite (if run in JupyterLite)
#    import piplite
#    await piplite.install(['numpy'])
#except ImportError:
#    pass

In [ ]:
import random
#import numpy as np
import math

In [ ]:
load = [18, 5, 5, 5, 5, 1, 7, 3, 4, 22, 24, 25, 26, 27, 28, 27, 23, 21, 32, 27, 34, 24, 25, 20]
sum(load)/len(load)

In [ ]:
wind = [random.randint(2,15) for _ in range(24)]
#wind

In [ ]:
# Generate 12 points between 0 and pi (half of the sine wave)
#x = np.linspace(0, np.pi, 12)
sinusoidal_values = [math.sin(a/11*3.141) for a in range(12)]

# Apply the sine function to get a half-sinusoidal distribution
#sinusoidal_values = np.sin(x)

# Normalize the values to a specific range if needed (optional)
# For example, scaling between 0 and 1
normalized_values = [a/max(sinusoidal_values) for a in sinusoidal_values]

# Peak Solar power is 20KW
solar = [round(a * 20) for a in normalized_values]
solar = [0] * 6 + solar + [0] * 6
#solar

regenerate = [a + b for a, b in zip(wind, solar)]


In [ ]:
print(f'Hourly regenerative power available is: {regenerate}')

In [ ]:
print(f'Hourly load for a day is: {load}')
# Base load generator
nuclear = 6

# Total green power generation available as sum of wind, solar, nuclear
available_supply = [green + nuclear for green in regenerate]
print(f'Available green power in microgrid is: {available_supply}')

# Power deficit is load minus available power
deficit_power = [a - b for a,b in zip(load, available_supply)]
print(f'Deficit in microgrid is: {deficit_power}')

# Initialize battery requirement
soc = sum(available_supply) - sum(load)
battery_capacity = 0
#cap = []

for reqd_power in deficit_power:
    if reqd_power < 0:
        soc -= reqd_power
    else:  # Charge the battery (excess in power)
        soc = max(soc - reqd_power, 0)  # Ensure no negative charge
    # Track the maximum battery capacity needed
    battery_capacity = max(battery_capacity, soc)
    #cap.append(soc)

battery_capacity -= sum(available_supply) - sum(load)
# Output the maximum battery capacity required
print(f'The battery capacity required is: {battery_capacity} kWh')
#print(cap)

In [ ]:
# Introduce Storage to cater for deficit
print(f'Required Battery capacity: {battery_capacity}')
battery_charged = sum(available_supply) - sum(load)
soc = [round(battery_charged / battery_capacity, 2)]

wasted_power = []

for deficit in deficit_power:
    if deficit > 0:
        battery_charged -= deficit
        wasted_power.append(0)
    else:
        battery_charged -= deficit
        if battery_charged >= battery_capacity:
            wasted_power.append(battery_charged - battery_capacity)
            battery_charged = battery_capacity
        
    soc.append(round(battery_charged / battery_capacity, 2))

print(f'State of Battery Charge: {soc}')
print(f'Energy Wasted: {wasted_power}')

In [ ]:
print(f'Total load in a day: {sum(load)}')
print(f'Total generation in a day: {sum(available_supply)}')
print(f'Total of power not consumed: {sum(wasted_power)}')

In [ ]:
nuclear_max = 6
nuclear_minm = 4

battery_charged = sum(available_supply) - sum(load)
soc = [round(battery_charged / battery_capacity, 2)]

wasted_power = []


short = [a - b for (a,b) in zip(regenerate, load)]

In [ ]:
print(regenerate)
print(load)
print(short)

In [ ]:
# The goal is to simulate the power balance in a microgrid with solar, wind, nuclear, and battery storage based on a given load profile.
# We will follow these steps:
# 1. Adjust nuclear power to minimize excess energy when renewable generation exceeds load + battery charging capacity.
# 2. Track battery SOC (state of charge) throughout the day.
# 3. Ensure load is served by a combination of renewables, nuclear, and battery power.

# Given data
load_profile = [18, 5, 5, 5, 5, 1, 7, 3, 4, 22, 24, 25, 26, 27, 28, 27, 23, 21, 32, 27, 34, 24, 25, 20]
renewable_generation = [13, 15, 5, 4, 13, 15, 9, 15, 23, 30, 33, 22, 35, 24, 22, 15, 17, 4, 2, 13, 2, 5, 10, 9]
battery_capacity = 163  # Total battery capacity
battery_soc = 0.37 * battery_capacity  # Initial state of charge (37% of capacity)
battery_max_charge_rate = 10  # Assume a maximum charge rate of 10 units/hour
battery_max_discharge_rate = 10  # Assume a maximum discharge rate of 10 units/hour
nuclear_min = 4  # Minimum nuclear power
nuclear_max = 8  # Maximum nuclear power

# To store results
nuclear_power_output = []
battery_soc_history = []
excess_energy_history = []

for hour in range(24):
    # Calculate the initial required nuclear power assuming no excess
    nuclear_power = max(nuclear_min, min(nuclear_max, load_profile[hour] - renewable_generation[hour]))
    
    # Check the total available generation (renewable + nuclear)
    total_generation = renewable_generation[hour] + nuclear_power
    
    # If generation exceeds load and battery charging, adjust nuclear to minimize excess
    excess_energy = total_generation - load_profile[hour]
    
    if excess_energy > 0:
        # Calculate the amount of energy that can be stored in the battery
        available_battery_capacity = battery_capacity - battery_soc
        charge_to_battery = min(excess_energy, available_battery_capacity, battery_max_charge_rate)
        battery_soc += charge_to_battery
        excess_energy -= charge_to_battery
        
        # Adjust nuclear power to minimize the remaining excess energy
        if excess_energy > 0:
            nuclear_power -= min(excess_energy, nuclear_power - nuclear_min)
    
    # If generation is less than the load, discharge battery if possible
    elif total_generation < load_profile[hour]:
        deficit_energy = load_profile[hour] - total_generation
        discharge_from_battery = min(deficit_energy, battery_soc, battery_max_discharge_rate)
        battery_soc -= discharge_from_battery
        deficit_energy -= discharge_from_battery
        
        # If there is still a deficit after battery discharge, increase nuclear power if possible
        if deficit_energy > 0:
            nuclear_power = min(nuclear_max, nuclear_power + deficit_energy)
    
    # Store the results for each hour
    nuclear_power_output.append(nuclear_power)
    battery_soc_history.append(battery_soc)
    excess_energy_history.append(excess_energy)

# Convert results to numpy arrays for easier analysis
# nuclear_power_output = np.array(nuclear_power_output)
# battery_soc_history = np.array(battery_soc_history)
# excess_energy_history = np.array(excess_energy_history)

print(f' Nuclear Power Generated: {nuclear_power_output}')
print(f' Battery State of Charge: {battery_soc_history}')
print(f' Excess charge history: {excess_energy_history}')


In [ ]:
import matplotlib.pyplot as plt

# Time axis for the 24-hour period
hours = [i for i in range(len(load_profile))]

# Plot the load profile, renewable generation, and nuclear power output
plt.figure(figsize=(14, 8))

# Load Profile
plt.plot(hours, load_profile, label='Load Profile', color='black', linewidth=2, linestyle='--')
# Renewable Generation (Solar + Wind)
plt.plot(hours, renewable_generation, label='Renewable Generation (Solar + Wind)', color='green', linewidth=2)
# Nuclear Power Output
plt.plot(hours, nuclear_power_output, label='Nuclear Power Output', color='orange', linewidth=2)
# Battery SOC
plt.plot(hours, battery_soc_history, label='Battery SOC', color='blue', linewidth=2)

# Add grid, legend, and labels
plt.grid(True)
plt.legend(loc='upper left')
plt.xlabel('Hour of the Day')
plt.ylabel('Power (Units)')
plt.title('Microgrid Power Distribution and Battery SOC Over 24 Hours')
plt.xticks(hours)  # Mark every hour

# Show the plot
plt.tight_layout()
plt.show()


In [ ]:
x = [i for i in range(24)]
print(x)

In [ ]:
import numpy as np

# Hourly load and regenerative power data
hourly_load = np.array([18, 5, 5, 5, 5, 1, 7, 3, 4, 22, 24, 25, 26, 27, 28, 27, 23, 21, 32, 27, 34, 24, 25, 20])
regenerative_power = np.array([13, 6, 6, 9, 7, 14, 6.0, 15.0, 15.0, 24.0, 33.0, 25.0, 25.0, 33.0, 19.0, 19.0, 13.0, 4.0, 6, 8, 4, 2, 5, 9])

# Base load generator capacities
min_generator_output = 6  # kW
max_generator_output = 10  # kW

# Calculate energy deficit
energy_deficit = np.maximum(hourly_load - regenerative_power, 0)

# Calculate the energy contribution of the generator
generator_contribution = np.minimum(max_generator_output, energy_deficit)

# Adjust the generator output to cover the deficit within its range
remaining_deficit = np.maximum(energy_deficit - generator_contribution, 0)

# Calculate total remaining energy deficit (in kWh)
total_remaining_deficit = np.sum(remaining_deficit)  # Since each value is in kW and the duration is 1 hour

print(f"Total remaining energy deficit for the day: {total_remaining_deficit} kWh")

# Battery capacity is the remaining deficit that needs to be covered
battery_capacity = total_remaining_deficit
print(f"Required battery capacity: {battery_capacity} kWh")


In [ ]:
import numpy as np

# Hourly load and regenerative power data
hourly_load = np.array([18, 5, 5, 5, 5, 1, 7, 3, 4, 22, 24, 25, 26, 27, 28, 27, 23, 21, 32, 27, 34, 24, 25, 20])
regenerative_power = np.array([13, 6, 6, 9, 7, 14, 6.0, 15.0, 15.0, 24.0, 33.0, 25.0, 25.0, 33.0, 19.0, 19.0, 13.0, 4.0, 6, 8, 4, 2, 5, 9])

# Base load generator capacities
min_generator_output = 6  # kW
max_generator_output = 10  # kW

# Calculate energy deficit
energy_deficit = np.maximum(hourly_load - regenerative_power, 0)

# Calculate the generator's actual contribution for each hour
generator_contribution = np.minimum(max_generator_output, energy_deficit)
# Adjust generator output within its range
generator_contribution = np.maximum(generator_contribution, min_generator_output)

# Calculate the remaining deficit after accounting for the generator's contribution
remaining_deficit = np.maximum(energy_deficit - generator_contribution, 0)

# Calculate total remaining energy deficit (in kWh)
total_remaining_deficit = np.sum(remaining_deficit)  # Since each value is in kW and the duration is 1 hour

print(f"Total remaining energy deficit for the day: {total_remaining_deficit} kWh")

# Battery capacity is the remaining deficit that needs to be covered
battery_capacity = total_remaining_deficit
print(f"Required battery capacity: {battery_capacity} kWh")
